In [32]:
import pandas as pd

In [33]:
PATH_TO_JSON = 'trial_task.json'
raw_df = pd.read_json(PATH_TO_JSON)

In [34]:
raw_df.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [35]:
print('колличество строк без товаров:',raw_df[raw_df['products'].apply(lambda x: len(x) == 0)].shape)
print('незаполненные строки:',raw_df[raw_df.isna().any(axis=1)].shape)

колличество строк без товаров: (0, 4)
незаполненные строки: (0, 4)


In [36]:
cols_names= ['order_id','warehouse_name','highway_cost','product','price','quantity']

exploded_df = raw_df.explode('products')
full_df = pd.DataFrame(exploded_df['products'].tolist(), index=exploded_df.index)
full_df['order_id'] = exploded_df['order_id']
full_df['warehouse_name'] = exploded_df['warehouse_name']
full_df['highway_cost'] = exploded_df['highway_cost']
full_df = full_df.reindex(columns=cols_names).reset_index(drop=True)

full_df.head()

,order_id,warehouse_name,highway_cost,product,price,quantity
0,11973,Мордор,-70,ломтик июльского неба,450,1
1,11973,Мордор,-70,билет в Израиль,1000,3
2,11973,Мордор,-70,статуэтка Ленина,200,3
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1
4,85794,отель Лето,-50,зеленая пластинка,10,2


## 1. Найти тариф стоимости доставки для каждого склада

### Вычисления

In [37]:
def culc_tarif(row):
    total_quantity = 0
    for prodact in row['products']:
        total_quantity += prodact['quantity']
    tarif = -int(row['highway_cost']/total_quantity)
    return tarif
        

warehouse_df = raw_df.groupby('warehouse_name').head(1).copy()
warehouse_df['tarif'] = warehouse_df.apply(culc_tarif, axis=1)
warehouse_tarif_df = warehouse_df[['warehouse_name','tarif']].reset_index(drop=True)

### Итог

In [38]:
warehouse_tarif_df

,warehouse_name,tarif
0,Мордор,10
1,хутор близ Диканьки,15
2,отель Лето,25
3,остров невезения,5
4,гиперборея,20


## 2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')

### Вычисления

In [39]:
product_temp_df = full_df.copy()
product_temp_df['income'] = product_temp_df['price'] * product_temp_df['quantity']
product_df = product_temp_df.groupby('product', as_index=False).agg({
    'quantity': 'sum',
    'income': 'sum',
    'highway_cost': lambda x: -x.sum()
}).rename(columns={'highway_cost': 'expenses'})
product_df['profit'] = product_df['income'] - product_df['expenses']

### Итог

In [40]:
product_df

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,2090,26710
1,билет в Израиль,58,58000,2305,55695
2,зеленая пластинка,61,610,2260,-1650
3,ломтик июльского неба,42,18900,1210,17690
4,плюмбус,65,16250,2255,13995
5,подписка на suppi-блог,33,4950,1415,3535
6,статуэтка Ленина,68,13600,2235,11365


## Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

### Вычисления

In [41]:
temp_product_df = full_df.copy()
temp_product_df['posision_profit'] = temp_product_df['highway_cost'] + temp_product_df['price'] * temp_product_df[
    'quantity']

order_profit_df = temp_product_df[['order_id', 'posision_profit']].\
    groupby('order_id').sum().\
    rename(columns={"posision_profit": "order_profit"})
mean_order_profit = order_profit_df['order_profit'].mean()
del temp_product_df

### Итог

In [42]:
order_profit_df

,order_profit
order_id,
124,705
1391,490
2091,1200
2108,190
2558,205
...,...
98100,1580
98423,1170
99220,925


In [43]:
print('Средняя прибыль заказов:',mean_order_profit)

Средняя прибыль заказов: 1273.4


## 4. Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

### Вычисления 

In [44]:
temp_warehouse_df = full_df.copy()
temp_warehouse_df['posision_profit'] = temp_warehouse_df['highway_cost'] + temp_warehouse_df['price'] * \
                                       temp_warehouse_df['quantity']

grouped_df = temp_warehouse_df.groupby(['warehouse_name', 'product'], as_index=False).agg({
    'quantity': 'sum',
    'posision_profit': 'sum'
})
del temp_warehouse_df
warehouse_profit_df = grouped_df.groupby('warehouse_name')['posision_profit'].sum().reset_index()
warehouse_profit_df.rename(columns={'posision_profit': 'warehouse_profit'}, inplace=True)

merged_df = pd.merge(grouped_df, warehouse_profit_df, on='warehouse_name')
merged_df['percent_profit_product_of_warehouse'] = (merged_df['posision_profit'] / merged_df['warehouse_profit']) * 100
merged_df.rename(columns={'posision_profit': 'profit'}, inplace=True)
warehouse_df = merged_df[['warehouse_name', 'product', 'quantity', 'profit', 'percent_profit_product_of_warehouse']]
del warehouse_profit_df
del merged_df

### Итог

In [45]:
warehouse_df

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2290,13.895631
1,Мордор,билет в Израиль,9,8860,53.762136
2,Мордор,зеленая пластинка,11,-80,-0.485437
3,Мордор,ломтик июльского неба,3,1260,7.645631
4,Мордор,плюмбус,6,1360,8.252427
5,Мордор,подписка на suppi-блог,8,980,5.946602
6,Мордор,статуэтка Ленина,10,1810,10.983010
7,гиперборея,автограф Стаса Барецкого,12,6560,18.344519
8,гиперборея,билет в Израиль,21,20020,55.984340
9,гиперборея,зеленая пластинка,10,-480,-1.342282


## 5. Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться 'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

### Вычисления

In [46]:
grouped_by_warehouse_df = warehouse_df.groupby('warehouse_name',as_index=False).apply(lambda x: x.sort_values('percent_profit_product_of_warehouse', ascending=False))
grouped_by_warehouse_df['accumulated_percent_profit_product_of_warehouse'] = grouped_by_warehouse_df.groupby('warehouse_name', as_index=False)['percent_profit_product_of_warehouse'].cumsum()
grouped_by_warehouse_df = grouped_by_warehouse_df.reset_index(drop=True)

### Итог

In [47]:
# С группировкой по складам
grouped_by_warehouse_df

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,Мордор,билет в Израиль,9,8860,53.762136,53.762136
1,Мордор,автограф Стаса Барецкого,4,2290,13.895631,67.657767
2,Мордор,статуэтка Ленина,10,1810,10.983010,78.640777
3,Мордор,плюмбус,6,1360,8.252427,86.893204
4,Мордор,ломтик июльского неба,3,1260,7.645631,94.538835
5,Мордор,подписка на suppi-блог,8,980,5.946602,100.485437
6,Мордор,зеленая пластинка,11,-80,-0.485437,100.000000
7,гиперборея,билет в Израиль,21,20020,55.984340,55.984340
8,гиперборея,автограф Стаса Барецкого,12,6560,18.344519,74.328859
9,гиперборея,ломтик июльского неба,13,5390,15.072707,89.401566


## 6. Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). 
Если значение накопленного процента меньше или равно 70, то категория A.
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

### Вычисления

In [48]:
def set_category(row):
    if row['accumulated_percent_profit_product_of_warehouse'] <= 70:
        category = 'A'
    elif 70 < row['accumulated_percent_profit_product_of_warehouse'] <= 90:
        category = 'B'
    else:
        category = 'C'
    return category

grouped_by_warehouse_category_df = grouped_by_warehouse_df.copy()
grouped_by_warehouse_category_df['category'] = grouped_by_warehouse_category_df.apply(set_category, axis=1)   

### Итог

In [49]:
grouped_by_warehouse_category_df

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,Мордор,билет в Израиль,9,8860,53.762136,53.762136,A
1,Мордор,автограф Стаса Барецкого,4,2290,13.895631,67.657767,A
2,Мордор,статуэтка Ленина,10,1810,10.983010,78.640777,B
3,Мордор,плюмбус,6,1360,8.252427,86.893204,B
4,Мордор,ломтик июльского неба,3,1260,7.645631,94.538835,C
5,Мордор,подписка на suppi-блог,8,980,5.946602,100.485437,C
6,Мордор,зеленая пластинка,11,-80,-0.485437,100.000000,C
7,гиперборея,билет в Израиль,21,20020,55.984340,55.984340,A
8,гиперборея,автограф Стаса Барецкого,12,6560,18.344519,74.328859,B
9,гиперборея,ломтик июльского неба,13,5390,15.072707,89.401566,B
